Write a function $f(\delta, t)$ which counts the number of bi-directional edges in a $\delta$ interval starting at $t$.

1. Read graph in as a dataframe source, dest, timestamp
2. Add code to duplicate each edge timestamp with period input : i -> p + i
3. Go through the edges in sorted order from given input t till interval $\delta$.
4. Count number of edges in this bucket. $C_1$.
5. Sort by the destination node.
6. Drop duplicates.
7. Count the number of edges. $C_2$.
8. Number of bi-direc edges in ($\delta, t)$ is $C_1 - C_2$.

In [1]:
import pandas as pd
import numpy as np

In [2]:
graph_df = pd.read_csv('graph.txt', sep=" ")
print(graph_df)

   source  destination  timestamp
0       0            2          2
1       1            0          2
2       0            1          4
3       1            2          4
4       0            2          4
5       1            3          3


In [3]:
def dup_edges(graph_df, period = 12):
    """Duplicate each edge with new timestamps i > period+i"""    
    return pd.concat([graph_df, pd.concat([graph_df.iloc[:,[0, 1]], graph_df.iloc[:,[2]]+period], axis=1)], ignore_index=True)

In [4]:
def bi_direc_count(graph_df, delta, t):
    g_df_sorted = dup_edges(graph_df).sort_values(by= 'timestamp', ignore_index =True)
    iterator = g_df_sorted.iterrows()
    start_index = -1
    end_index = -1
    for row in iterator:
        if row[1]['timestamp'] >= t:
            start_index = row[0]
            break
    for row in iterator:
        if row[1]['timestamp'] <= delta + t:
            end_index = row[0]
    print(start_index, end_index)
    if start_index == -1:
        print('None')
        return
    seq_df = g_df_sorted.iloc[start_index:end_index]
    c_1 = len(seq_df.index)
    seq_list_2 = []
    for row in seq_df.iterrows():
        if row[1]['source'] > row[1]['destination']:
            d = pd.Series({'source': row[1]['destination'], 
                       'destination': row[1]['source'], 
                       'timestamp': row[1]['timestamp']})
        else:
            d = row[1]
        seq_list_2.append(d)
    seq_df_2 = pd.concat(seq_list_2, axis=1).transpose()
    print(seq_df_2)
    c_2 = len(seq_df_2.drop_duplicates(subset = ['source', 'destination']).index)
    return c_1 - c_2


bi_direc_count(graph_df, 20, 2)


0 11
    source  destination  timestamp
0        0            2          2
0        0            1          2
2        1            3          3
3        0            1          4
4        1            2          4
5        0            2          4
6        0            2         14
1        0            1         14
8        1            3         15
9        0            1         16
10       1            2         16


7

Edges in same direction are being counted.